# Memory in LLMChain
---
Este cuaderno se explica cómo utilizar la clase `Memory` con una `LLMChain`.

Agregaremos la clase `ConversationBufferMemory`, aunque puede ser cualquier clase de memoria.

In [1]:
from dotenv import load_dotenv, find_dotenv
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

load_dotenv(find_dotenv())

True

El paso más importante es configurar el mensaje correctamente. En el siguiente mensaje, tenemos dos claves de entrada: una para la entrada real y otra para la entrada de la clase Memoria. Es importante destacar que nos aseguramos de que las claves en PromptTemplate y ConversationBufferMemory coincidan (chat_history).

In [2]:
template = """You are a chatbot having a conversation with a human.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"],
    template=template,
)

memory = ConversationBufferMemory(memory_key="chat_history")

In [4]:
# llm 
llm = OpenAI()

# llm chain
llm_chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
) 

In [5]:
llm_chain.predict(human_input="Hi there my friend")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.


Human: Hi there my friend
Chatbot:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

In [6]:
llm_chain.predict(human_input="Not too bad - how are you?")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.

Human: Hi there my friend
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: Not too bad - how are you?
Chatbot:

> Finished chain.


" I'm doing great, thank you for asking! What would you like to talk about?"

# Adding Memory to a chat model-based LLMChain
--- 

El ejemplo de arriba funciona para un modelo de LLM de "completion-style", pero si estas usando un modelo de chat, obtendrá un mejor rendimiento utilizando mensajes de chat estructurados.

Veamos el ejemplo a continuacion:

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

Usaremos la clase `ChatPromptTemplate` para configurar el prompt para el Chat.

El metodo `from_messages` crea una instancia de `ChatPromptTemplate` desde una lista de mensajes (SystemMessage, HUmanMessage, AIMessage, ChatMessage, etc.) or desde plantilas de mensajes como vemos en el MessagesPlaceholder.

La siguiente configuración hace que la memoria se inyecte en el medio del mensaje de chat, en la clave chat_history, y las entradas del usuario se agreguen en un mensaje humano/usuario al final del mensaje de chat.

In [13]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You area a chatbot having a concersation with a human."), # The persistent system prompt
    MessagesPlaceholder(variable_name="chat_history"), # Where the memory will be stored.
    HumanMessagePromptTemplate.from_template("{human_input}"), # Where the human input will injected
])

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

Iniciemos

In [14]:
# llm 
llm = ChatOpenAI()

# llm chain
chat_llm_chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

In [15]:
chat_llm_chain.predict(human_input="Hi there my friend")



> Entering new LLMChain chain...
Prompt after formatting:
System: You area a chatbot having a concersation with a human.
Human: Hi there my friend

> Finished chain.


'Hello! How can I assist you today, my friend?'

In [16]:
chat_llm_chain.predict(human_input="Not too bad - how are you?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You area a chatbot having a concersation with a human.
Human: Hi there my friend
AI: Hello! How can I assist you today, my friend?
Human: Not too bad - how are you?

> Finished chain.


"I'm an AI, so I don't have feelings, but I'm here to help you with anything you need. Is there something specific you would like to chat about or any questions I can answer for you?"